In [2]:
# Imports + Load

import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\User\OneDrive\Documents\OR PROJECTS\data\raw\ab_test_data.csv")
control = df[df["group"] == "control"].copy()
treatment = df[df["group"] == "treatment"].copy()


In [4]:
# ATE (Difference-in-means)

ate_conv = treatment["converted"].mean() - control["converted"].mean()
ate_rev = treatment["revenue"].mean() - control["revenue"].mean()

print(f"ATE (Conversion): {ate_conv:.4f}")
print(f"ATE (Revenue):    {ate_rev:.2f}")


ATE (Conversion): 0.0314
ATE (Revenue):    9.07


In [5]:
# Bootstrap CI for ATE (conversion + revenue)

def bootstrap_ate(df, metric_col, group_col="group", treat_label="treatment", control_label="control", B=2000, seed=42):
    rng = np.random.default_rng(seed)
    t = df[df[group_col] == treat_label][metric_col].to_numpy()
    c = df[df[group_col] == control_label][metric_col].to_numpy()

    ates = np.empty(B)
    for b in range(B):
        t_s = rng.choice(t, size=len(t), replace=True)
        c_s = rng.choice(c, size=len(c), replace=True)
        ates[b] = t_s.mean() - c_s.mean()

    lo, hi = np.quantile(ates, [0.025, 0.975])
    return ates.mean(), lo, hi

mean_ate_c, lo_c, hi_c = bootstrap_ate(df, "converted", B=2000)
mean_ate_r, lo_r, hi_r = bootstrap_ate(df, "revenue", B=2000)

print(f"Bootstrap 95% CI ATE Conversion: [{lo_c:.4f}, {hi_c:.4f}] (mean={mean_ate_c:.4f})")
print(f"Bootstrap 95% CI ATE Revenue:    [{lo_r:.2f}, {hi_r:.2f}] (mean={mean_ate_r:.2f})")


Bootstrap 95% CI ATE Conversion: [0.0190, 0.0432] (mean=0.0313)
Bootstrap 95% CI ATE Revenue:    [7.83, 10.32] (mean=9.07)


In [6]:
# Causal decision framing (target rollout)

conv_control = control["converted"].mean()
conv_treat = treatment["converted"].mean()

print(f"Control conversion:   {conv_control:.4f}")
print(f"Treatment conversion: {conv_treat:.4f}")
print(f"Absolute lift:        {(conv_treat - conv_control):.4f}")
print(f"Relative lift:        {((conv_treat/conv_control)-1):.2%}")


Control conversion:   0.0958
Treatment conversion: 0.1272
Absolute lift:        0.0314
Relative lift:        32.78%


Causal Interpretation:
Because users are randomly assigned to Control vs Treatment, the difference in average outcomes is an unbiased estimate of the Average Treatment Effect (ATE).

Why bootstrap?
Revenue can be skewed; bootstrap confidence intervals provide a robust uncertainty estimate without relying heavily on normality assumptions.

Recommendation:
If conversion lift is positive and the revenue impact is favorable (and operational risk is low), roll out the new onboarding flow. Continue monitoring longer-term metrics such as retention and customer support volume.